#OAuth Exercise

In this exercise we will try to scrape twitter data and do a tf-idf analysis on that (src-uwes twitter analysis). We will need OAuth authentication, and we will follow a similar approach as detailed in the yelp analysis notebook. 

In [1]:
import oauth2 as oauth
import urllib2 as urllib

In [2]:
oauth.OAUTH_VERSION

'1.0'

We will now need twitter api access. The following steps as available online will help you set up your twitter account and access the live 1% stream.

1. Create a twitter account if you do not already have one.
2. Go to https://dev.twitter.com/apps and log in with your twitter credentials.
3. Click "Create New App"
4. Fill out the form and agree to the terms. Put in a dummy website if you don't have one you want to use.
5. On the next page, click the "API Keys" tab along the top, then scroll all the way down until you see the section "Your Access Token"
6. Click the button "Create My Access Token". You can Read more about Oauth authorization online. 

Save the details of api_key, api_secret, access_token_key, access_token_secret in your vaule directory and load it in the notebook as shown in yelpSample notebook.

In [3]:
# We need to define the following variables
#api_key = #<get api key> 
#api_secret = #<get api secret>
#access_token_key = #<get your access token key here>"
#access_token_secret = #<get your access token secret here>

#defining them right here is not safe. insteadm create a file in a different directory
# (I use ~/VaultDSE) and in it put a file called, say, twitterkeys.py whose
# content is:
#api_key = #<get api key>
#api_secret = #<get api secret>
#access_token_key = #<get your access token key here>"
#access_token_secret = #<get your access token secret here>
#
#def getkeys():
#    return api_key,api_secret,access_token_key,access_token_secret


# then use the following commands

import sys
sys.path.append('/Users/Deepthi/Documents/DSE/dmysoren/Python Day 5/VaultDSE')
import twitterKeys
api_key,api_secret,access_token_key,access_token_secret=twitterKeys.getkeys()

_debug = 0

oauth_token    = oauth.Token(key=access_token_key, secret=access_token_secret)
oauth_consumer = oauth.Consumer(key=api_key, secret=api_secret)

signature_method_hmac_sha1 = oauth.SignatureMethod_HMAC_SHA1()

http_method = "GET"

http_handler  = urllib.HTTPHandler(debuglevel=_debug)
https_handler = urllib.HTTPSHandler(debuglevel=_debug)

Below is a twitter request method which will use the above user logins to sign, and open a twitter stream request

In [4]:
def getTwitterStream(url, method, parameters):
    req = oauth.Request.from_consumer_and_token(oauth_consumer,
                                             token=oauth_token,
                                             http_method=http_method,
                                             http_url=url, 
                                             parameters=parameters)

    req.sign_request(signature_method_hmac_sha1, oauth_consumer, oauth_token)
    headers = req.to_header()

    if http_method == "POST":
            encoded_post_data = req.to_postdata()
    else:
        encoded_post_data = None
        url = req.to_url()

    opener = urllib.OpenerDirector()
    opener.add_handler(http_handler)
    opener.add_handler(https_handler)

    response = opener.open(url, encoded_post_data)

    return response


We can use the above function to request a response as follows

In [5]:
#Now we will test the above function for a sample data provided by twitter stream here -  
url = "https://stream.twitter.com/1/statuses/sample.json"

In [6]:
parameters = []
response = getTwitterStream(url, "GET", parameters)

Write a function which will take a url and return the top 10 lines returned by the twitter stream

** Note ** The response returned needs to be intelligently parsed to get the text data which correspond to actual tweets. This part can be done in a number of ways and you are encouraged to try different approaches to parse the response data.

In [7]:
## Get top 10 tweets returned by the twitter stream

## This is a simple fetchData() function which can be used to get the top 10 tweets without any criteria. 
## This is implemented using the url given above

import json

def fetchData(url):
    i=0
    parameters = []
    response = getTwitterStream(url, "GET", parameters)
    while(i<10):
        try:
            data = json.loads(response.readline())
            print i+1, data['text'], '\n'
            i+=1
        except:
            continue

fetchData("https://stream.twitter.com/1/statuses/sample.json")

1 awesome https://t.co/mEXtLGgmPw 

2 2 2 2 2 2 受話器越しになると照れるけど そりゃ一緒がいい 当たり前さ 思うほど上手くいかないけど 遠く遠く 離れ離れ 電話切れない夜もある 闇深く心細く それでも光射し 泣いたり笑ったりさ 別の街に暮らす君よ 寄り添ってやれないが 僕はここにいる&lt;♪電話/レミオロメン&gt; 

3 RT @AlDubBarkads: Eto pa! Pak! Fresh look! (©Mr Andy Sylvia's Fb) OMG! Diko kinakaya! Bagay na bagay! ❤ #ALDUBPromiseRing https://t.co/QAHY… 

4 RT @TheKateWolff: Please don't make a duck face in your breast feeding picture. 

5 RT @KatyTurNBC: "Most of the media is absolute scum" - Trump 

6 朝充電100%で朝からそんな携帯使ってないのにもう45%やねんけど 

7 RT @OneDrecti0nFans: "We're like....5 best friends" 

I AM GONNA CRY :')

#MTVStars One Direction 

https://t.co/lZkGrI25QQ 

8 ペース配分という言葉を知らないので改修工廠出して即12.7cm高角砲+高射装置作ってネジ使い果たした 

9 Se dizem amigos mais não me respondem, chega glr 

10 RT @Natynevees: @AnabelBezerra @any_sacana era de vcs sim 



In [9]:
# We can also request twitter stream data for specific search parameters as follows
# url= "https://api.twitter.com/1.1/search/tweets.json?q="+search_query

Call the fetchData function to fetch latest live stream data for following search queries and output the first 5 lines

1. "UCSD"
2. "Donald Trump"
3. "Syria"

In [10]:
## This is a slightly modified fetchdata function which will fetch the data and extract the tweet from a url with
## a search query. (the url given above is used)

import json

def fetchData(url):
    parameters = []
    response = getTwitterStream(url, "GET", parameters)
    data = json.loads(response.readline())
    print 'FOLLOWING ARE THE TOP 5 SEARCH QUERIES:'
    for i in xrange(int(cnt)):
        print i+1, data['statuses'][i]['text'],'\n'
        

search_query = ['UCSD','Syria','Donald Trump']
cnt = '5'
for search_query in search_query:
    url= "https://api.twitter.com/1.1/search/tweets.json?q="+search_query+'&'+'count='+cnt
    print '*******URL generated to obtain results for', search_query, url, '\n'
    fetchData(url)

*******URL generated to obtain results for UCSD https://api.twitter.com/1.1/search/tweets.json?q=UCSD&count=5 

FOLLOWING ARE THE TOP 5 SEARCH QUERIES:
1 UCSD student leaders criticized for cutting media funds: UCSD student leaders criticized for cutting funds to the… https://t.co/IlQYWLSPPP 

2 RT @arankhanna: Interesting (albeit quite technical) paper from @UCSDJacobs on how to de-anonymize users with webGL fingerprinting https://… 

3 RT @arankhanna: Interesting (albeit quite technical) paper from @UCSDJacobs on how to de-anonymize users with webGL fingerprinting https://… 

4 UCSD student leaders are facing some backlash for cutting funding to the Koala and other campus publications. https://t.co/Z3pkiGtWDj 

5 RT @UCSDtritons: #UCSD #AOTW are @ucsd_wvb Amanda Colla and @UCSDTritonsMWP Chase Cockerill.🔱 https://t.co/jMBfZS7kYt @USECreditUnion https… 

*******URL generated to obtain results for Syria https://api.twitter.com/1.1/search/tweets.json?q=Syria&count=5 

FOLLOWING ARE THE 

### TF-IDF###

tf–idf, short for term frequency–inverse document frequency, is a numerical statistic that is intended to reflect how important a word is to a document in a collection or corpus.It is among the most regularly used statistical tool for word cloud analysis. You can read more about it online (https://en.wikipedia.org/wiki/Tf%E2%80%93idf)

We base our analysis on the following

1. The weight of a term that occurs in a document is simply proportional to the term frequency
2. The specificity of a term can be quantified as an inverse function of the number of documents in which it occurs

For this question we will perform tf-idf analysis o the stream data we retrieve for a given search parameter. Perform the steps below

1. use the twitterreq function to search for the query "syria" and save the top 200 lines in the file twitterStream.txt
2. load the saved file and output the count of occurrences for each term. This will be your term frequency
3. Calculate the inverse document frequency for each of the term in the output above.
4. Multiply the term frequency for each of the term by corresponding inverse document frequency.
5. Sort the terms in the descending order based on their term freq/inverse document freq scores 
6. Print the top 10 terms.

In [11]:
## fetchData() function below fetches the data from the url which is constructed using search query and the counts
## In the example below, on 100 tweets are extracted as per clarification on piazza

import json

def fetchData(url):
    parameters = []
    response = getTwitterStream(url, "GET", parameters)
    data = json.loads(response.readline())
    newfile = open("twitterStream.txt","w")
    for i in xrange(int(cnt)):
        tmp = data['statuses'][i]['text']
        tmp2 = tmp.encode('utf-8')
        newfile.write(tmp2.replace('\n',' ') +'\n')

search_query = 'syria'
cnt = '100' ## Getting 100 tweets as per the clarification on Piazza
url= "https://api.twitter.com/1.1/search/tweets.json?q="+search_query+'&'+'count='+cnt
twitterStream = fetchData(url)


In [12]:
## tf-idf calculation

import pandas as pd

# Get the data from the extracted file
text = ! cat twitterStream.txt

# Calculate term frequency - tf
word_frequency = []
for i in range(len(text)):
    text[i]=str(text[i])
    for char in '-.,[]:\'{}|?(")_;<>=!*`':
        text[i]=text[i].replace(char,'')    
    for char in '\\':
        text[i]=text[i].replace(char,'')
    for char in '\n':
        text[i]=text[i].replace(char,'')
    text[i] = text[i].lower()
    words = text[i].split()
    count1 = {i:words.count(i) for i in set(words)}
    new = pd.DataFrame(count1.items(),columns =(['word','frequency']))
    word_frequency.append(new)
    
word_frequency2 = pd.concat(word_frequency, axis=0)


words = set(word_frequency2['word'])

# Calculate document frequency 
cnt = 0
ne = 0 
w_cnt = []
for w in words:
    with open("twitterStream.txt","r") as f:
        for line in f:
            line = line.lower()
            for char in '-.,[]:\'{}|?(")_;<>=!*`':
                line=line.replace(char,'')
            for char in '\\':
                line=line.replace(char,'')
            for char in '\n':
                line=line.replace(char,'')
            if w in line:
                cnt=1
                w_cnt.append((w,cnt,line))
            else:
                ne+=1
        
document_freq = pd.DataFrame(w_cnt, columns=(['word','cnt','line']))

document_freq2 = document_freq.groupby('word').agg('sum')

document_freq2.sort(['cnt'],ascending=0)


## Calculate inverse document frequency
import math

document_freq2['inv_doc']= document_freq2['cnt'].apply(lambda x: math.log(100/(1+x)))

result = pd.merge(word_frequency2, document_freq2.reset_index(), how='inner',on=['word','word'])


result['tf_idf'] = result['frequency']*result['inv_doc']

final_result = result[['word','tf_idf']]

## Get the top 10 terms used
final_result.groupby('word').agg({'tf_idf':'mean'}).sort('tf_idf',ascending = False).head(10)


## If you look at the results it looks like most of the words are those that add value to the tweet except 
## the word 'thats'. I think that since we are looking into tweets, very few people use the full form of these common 
## words and hence this is one off case. 
## Also, if you observe, the word 'Syria' doesn't appear in the top list. This implies that it doesn't add more value 
## into the document as it is present across all the documents. Thus proving the purpose of tf-idf


,tf_idf
word,
terrorism/genocide,7.824046
thats,7.824046
stabbing,7.824046
disgust,7.824046
ولا,7.824046
money,7.824046
again,6.993015
#buybooks,6.993015
terrorism,5.991465
